In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_867817/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 17:29:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 17:29:40 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [17]:
import pandas as pd

# Sample data for the Calls table
calls_data = {
    'caller_id': [8, 4, 5, 8, 11, 8, 4],
    'recipient_id': [4, 8, 1, 3, 3, 11, 12],
    'call_time': [
        '2021-08-24 17:46:07',
        '2021-08-24 19:57:13',
        '2021-08-11 05:28:44',
        '2021-08-17 04:04:15',
        '2021-08-17 13:07:00',
        '2021-08-17 22:22:22',
        '2021-08-24 22:22:22',
    ]
}

# Convert data to DataFrame
calls_df = pd.DataFrame(calls_data)

# Display the DataFrame
print("Calls table:")
print(calls_df)



df_person = spark.createDataFrame(calls_df)
df_person.createOrReplaceTempView("Calls")



Calls table:
   caller_id  recipient_id            call_time
0          8             4  2021-08-24 17:46:07
1          4             8  2021-08-24 19:57:13
2          5             1  2021-08-11 05:28:44
3          8             3  2021-08-17 04:04:15
4         11             3  2021-08-17 13:07:00
5          8            11  2021-08-17 22:22:22
6          4            12  2021-08-24 22:22:22


In [21]:
query = """
    with t as (
        select
            user_id,
            call_with_id,
            call_time,
            date(call_time) as date,
            call_time = min(call_time) over (partition by user_id, date(call_time)) as is_min,
            call_time = max(call_time) over (partition by user_id, date(call_time)) as is_max
        from (
            select caller_id as user_id, recipient_id as call_with_id, call_time from Calls
            union all
            select recipient_id as user_id, caller_id as call_with_id, call_time from Calls
        ) order by user_id, call_time, call_with_id
    )
    
    select
        distinct
        t_min.user_id
    from (
        select user_id, call_with_id, date from t where is_min
    ) t_min inner join (
        select user_id, call_with_id, date from t where is_max
    ) as t_max 
        on t_min.user_id = t_max.user_id
        and t_min.call_with_id = t_max.call_with_id
        and t_min.date = t_max.date
"""

In [22]:
spark.sql(query).show()

+-------+
|user_id|
+-------+
|      5|
|      1|
|     12|
|      8|
+-------+

